## Using Gala's reflex_correct with a frame loaded from Galstreams catalog

### Printing versions

In [1]:
from importlib.metadata import version
print("Python version: 3.10.15")
print("gala version:", version("gala"))
print("galstreams version:", version("galstreams"))

Python version: 3.10.15
gala version: 1.9.2.dev5
galstreams version: 1.0.2.dev294


### Importing packages

In [2]:
import numpy as np
import galstreams
import gala.coordinates as galacoord
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u

### Usage

#### Loading GreatCircleICRSFrame object associated to GD-1 from Galstreams

In [3]:
mwsts = galstreams.MWStreams(verbose=False, implement_Off=True)
stream = mwsts["GD-1-PB18"]
frame = stream.stream_frame

Initializing galstreams library from master_log... 


#### Loading Gaia DR3 data, previously downloaded from CATS repo.

In [4]:
data_file = "/home/mmestre/casa/work/Data/cats/Gaia/DR3/corr/GaiaDR3-GD-1-all_extincorr.fits"
df = at.Table.read(data_file)

#### Build SkyCoord object in ICRS frame (with fake distances and radial velocities)

In [5]:
stream_icrs = coord.SkyCoord(ra=df["ra"]*u.deg, dec=df["dec"]*u.deg, pm_ra_cosdec=df["pmra"]*u.mas/u.yr,
                           pm_dec=df["pmdec"]*u.mas/u.yr,
                           distance=np.ones(len(df))*u.kpc,
                           radial_velocity=np.zeros(len(df))*u.km/u.s, frame="icrs")

#### Transform to GreatCircleICRSFrame (sc = self stream coordinates)

In [6]:
stream_sc = stream_icrs.transform_to(frame)

#### Define the Galactocentric frame 

In [7]:
v_sun = coord.CartesianDifferential([11.1, 220.0+12.24, 7.25]*u.km/u.s)
r_sun = 8.122*u.kpc
gc_frame = coord.Galactocentric(galcen_distance=r_sun, galcen_v_sun=v_sun, z_sun=0*u.pc)

#### Perform the reflex correction. This first time passing the SkyCoord object in the ICRS frame. It works!

In [8]:
stream_icrs_reflexcorr = galacoord.reflex_correct(stream_icrs, gc_frame)
stream_icrs_reflexcorr

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(134.17858139,  0.69990361, 1.), (134.19378251,  0.72031546, 1.),
     (134.19509489,  0.72372441, 1.), ..., (132.56727591, -2.07624214, 1.),
     (132.55693954, -2.07754112, 1.), (132.5677011 , -2.06823086, 1.)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [( 1.34732154, 34.03479394, -155.12834339),
     (-7.71759277, 39.8813443 , -155.0659438 ),
     (-8.84169936, 28.1453817 , -155.05557198), ...,
     (-0.88280418, 32.4029197 , -163.34635196),
     (-3.8480967 , 37.54130603, -163.34973981),
     (-8.61758972, 37.66049476, -163.32320988)]>

#### Perform de reflex correction. This time passing the SkyCoord object in the GreatCircleICRSFrame. 
#### Could not make it work:

In [126]:
stream_sc_reflexcorr = galacoord.reflex_correct(stream_sc, gc_frame)

AttributeError: 'GreatCircleICRSFrame' object has no attribute '_R'

In [14]:
stream_icrs_reflexcorr.transform_to(frame)

<SkyCoord (GreatCircleICRSFrame: pole=<ICRS Coordinate: (ra, dec) in deg
    (34.5987, 29.7331)>, origin=<ICRS Coordinate: (ra, dec) in deg
    (200.48591244, 59.5048423)>, priority=origin): (phi1, phi2, distance) in (deg, deg, kpc)
    [(-77.49584072, -7.95776449, 1.), (-77.47030899, -7.96064744, 1.),
     (-77.46666724, -7.96006863, 1.), ..., (-80.73673826, -7.94763181, 1.),
     (-80.74305056, -7.93930694, 1.), (-80.72950184, -7.94402681, 1.)]
 (pm_phi1_cosphi2, pm_phi2, radial_velocity) in (mas / yr, mas / yr, km / s)
    [(30.10222369, 15.93858849, -155.12834339),
     (30.5998157 , 26.7158032 , -155.0659438 ),
     (19.88858682, 21.78949915, -155.05557198), ...,
     (27.69757559, 16.83962156, -163.34635196),
     (30.68853319, 21.96295602, -163.34973981),
     (28.42553412, 26.16418789, -163.32320988)]>

#### Once the previous cell is corrected, this next cell is to check the correctness of the result.

In [ ]:
error_phi1 = stream_icrs_reflexcorr.transform_to(frame).pm_phi1_cosphi2.value - stream_sc_reflexcorr.pm_phi1_cosphi2.value
error_phi2 = stream_icrs_reflexcorr.transform_to(frame).pm_phi2.value - stream_sc_reflexcorr.pm_phi2.value
print(error_phi1.max(), error_phi2.max())

#### A hack that strangely worked in Julia: instead of using reflex_correct(), run each command inside reflex_correct step by step. 
#### Now using the SkyCoords object in the GreatCircle frame that caused problem previously. It does not work either.

In [129]:
observed = stream_sc.transform_to(gc_frame)   #This is the line that has the problem
# rep = observed.cartesian.without_differentials()
# rep = rep.with_differentials(observed.cartesian.differentials["s"] + v_sun)
# stream_sc_reflexcorr_steps = gc_frame.realize_frame(rep).transform_to(frame)

AttributeError: 'GreatCircleICRSFrame' object has no attribute '_R'

#### To check that the previous hack works, not yet:

In [ ]:
error_phi1 = stream_icrs_reflexcorr.transform_to(frame).pm_phi1_cosphi2.value - stream_sc_reflexcorr_steps.pm_phi1_cosphi2.value
error_phi2 = stream_icrs_reflexcorr.transform_to(frame).pm_phi2.value - stream_sc_reflexcorr_steps.pm_phi2.value
print( "errors =", error_phi1.max(), error_phi2.max())